# Generate Jobs Report Data for Chartbook

Brian Dew

@bd_econ

In [1]:
import sys
import json
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

### API Request

In [41]:
# Series stored as a dictionary
series = {'LNS14000003': 'White', 
          'LNS14000006': 'Black', 
          'LNS14000009': 'Hispanic',
          'LNS14000000': 'Total',
          'LNS13000000': 'Level',
          'LNU03008636': 'LT',
          'LNU00000000': 'POP',
          'LNS12300060': 'PA_EPOP',
          'LNS14023621': 'Job Loser', 
          'LNS14023705': 'Job Leaver', 
          'LNS14023557': 'Re-entrant',
          'LNS14023569': 'New entrant',
          'LNS13008276': 'Median',
          'LNS13008275': 'Mean'}

# Start year and end year
dates = (1988, 2020)
df = bls_api(series, dates, bls_key)
df.to_csv(data_dir / 'jobs_report_main.csv', index_label='date')

Post Request Status: REQUEST_SUCCEEDED


### Unemployment rate

In [5]:
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', parse_dates=['date'])
        .set_index('date'))
srs = ['White', 'Black', 'Hispanic']
df.loc['1989':, srs].to_csv(data_dir / 'unemp.csv', index_label='date')

s = series_info(df['Level'])
s2 = series_info(df['Total'])
s3 = series_info(df['Black'])
text = ('Unemployment is currently very low. BLS '+
        '\href{https://www.bls.gov/news.release/empsit.nr0.htm}{reports} '+
        f'{s["val_latest"]/1000:.1f} million '+
        f'unemployed persons in {s["date_latest_ft"]}, '+
        f'and an unemployment rate of {s2["val_latest"]} percent. '+
        'However, unemployment is much higher for disadvantaged groups, '+
        'with the black or African American unemployment rate typically double '+
        'the white unemployment rate. '+
        'A very tight labor market may have the effect of reducing racial '+
        'discrimination in hiring. '+
        'Over the past year, the black or African American unemployment rate '+
        f'has fallen by {abs(s3["change_year_ago"]):.1f} percentage '+
        f'points to {s3["val_latest"]:.1f} percent.')
write_txt(text_dir / 'unemp.txt', text)
print(text)

Unemployment is currently very low. BLS \href{https://www.bls.gov/news.release/empsit.nr0.htm}{reports} 5.9 million unemployed persons in January 2020, and an unemployment rate of 3.6 percent. However, unemployment is much higher for disadvantaged groups, with the black or African American unemployment rate typically double the white unemployment rate. A very tight labor market may have the effect of reducing racial discrimination in hiring. Over the past year, the black or African American unemployment rate has fallen by 0.8 percentage points to 6.0 percent.


### Employment rate

In [6]:
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', parse_dates=['date'])
        .set_index('date')).loc['1989':, 'PA_EPOP']

node = f'{dtxt(df.index[-1])["mon5"]}:\\\{df.iloc[-1]:.1f}\%'
write_txt(text_dir / 'epop.txt', node)

d = series_info(df)

text = f'In {d["date_latest_ft"]}, {d["val_latest"]}\%'

if (d['days_since_match'] > 725) | (d['days_since_match'] == 0):
    text2 = d['last_matched']
else:
    text2 = f'compared to {d["val_prev"]} in {d["date_prev_ft"]}'
    
if d['change_year_ago'] > 0:
    direction = 'increased'
    value = d['change_year_ago']
    label = (f'Over the past year, the age 25-54 employment rate has '+
             f'{direction} by {value:.1f} percentage points.')
elif d['change_year_ago'] < 0:
    direction = 'fallen'
    value = abs(d['change_year_ago'])
    label = (f'Over the past year, the age 25-54 employment rate has '+
             f'{direction} by {value:.1f} percentage points.')
else:
    label = 'The age 25-54 employment rate is unchanged over the past year. '
    
pop = 126277
diff1 = d['late90s'] - d['val_latest']
diff = (d['late90s'] - d['val_latest']) / 100 * pop

if diff > 999:
    diff_text = f'{round(diff / 1000, 1)} million'
else: 
    diff_text = f'{round(diff, -1)} thousand'
    
label2 = (f'The current age 25-54 employment rate is {diff1:.1f} percentage '+
          f'points (equivalent to {diff_text} workers) below the average during '+
          '1998--99, a period with a particularly tight labor market. ')

textval = f'{text} of 25-54 years olds were employed, {text2}. {label} {label2}'
print(textval)

write_txt(text_dir / 'epop_text.txt', textval)

In January 2020, 80.6\% of 25-54 years olds were employed, the highest level since June 2001. Over the past year, the age 25-54 employment rate has increased by 0.8 percentage points. The current age 25-54 employment rate is 0.7 percentage points (equivalent to 840.0 thousand workers) below the average during 1998--99, a period with a particularly tight labor market. 


### Unemployment by reason

In [8]:
srs = ['Job Loser', 'Job Leaver', 'Re-entrant', 'New entrant']
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', parse_dates=['date'])
        .set_index('date')).loc['1989':, srs]

df.resample('QS').mean().to_csv(data_dir / 'unemp_reason.csv', index_label='date', float_format='%g')

loser = df['Job Loser'].iloc[-1]
leaver = df['Job Leaver'].iloc[-1]
reent = df['Re-entrant'].iloc[-1]
newent = df['New entrant'].iloc[-1]
ltdate = df.index[-1].strftime('%B %Y')

text = (f'In {ltdate}, {loser:.1f} percent of the labor force '+
        'were unemployed because of losing a job or having a '+
        f'job end, {leaver:.1f} percent were re-entrants, '+
        f'{reent:.1f} percent new entrants, and {newent:.1f} '+
        'percent job leavers. ')

write_txt(text_dir / 'unemp_reason.txt', text)

text

'In January 2020, 1.6 percent of the labor force were unemployed because of losing a job or having a job end, 0.5 percent were re-entrants, 1.1 percent new entrants, and 0.3 percent job leavers. '

### Unemployed long-term

In [43]:
srs = ['LT', 'POP']
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', parse_dates=['date'])
        .set_index('date')).loc['1989':, srs]

data = ((df['LT'] / df['POP']) * 100)
data.to_csv(data_dir / 'ltu.csv', index_label='date', header=['VALUE'], float_format='%g')

write_txt(text_dir / 'ltu_node.txt', end_node(data, 'blue'))

ldate = dtxt(data.index[-1])['mon1']
pdate = dtxt(data.index[-13])['mon1']
hdate = dtxt(data.idxmax())['mon1']

text = (f'As of {ldate}, BLS '+
        '\href{https://www.bls.gov/webapps/legacy/cpsatab12.htm}{reports} '+
        f'that {data.iloc[-1]:.2f} percent of the age 16+ '+
         'population have been unemployed for 27 weeks or longer, '+
        f'compared to {data.iloc[-13]:.2f} percent in {pdate}. '+
        f'Long-term unemployment peaked at {data.max():.2f} percent of '+
        f'the population in {hdate}.')

write_txt(text_dir / 'ltu.txt', text)

### Duration of Unemployment

In [45]:
srs = ['Median', 'Mean']
df = (pd.read_csv(data_dir / 'jobs_report_main.csv', parse_dates=['date'])
        .set_index('date')).loc['1989':, srs]

df.to_csv(data_dir / 'unempdur.csv', index_label='date', float_format='%g')

ldate = dtxt(df.index[-1])['mon1']

median = df['Median'].iloc[-1]
mean = df['Mean'].iloc[-1]

text = ('More concerning, however, is that among those who are unemployed '+
        f'the average (mean) duration of unemployment is {mean:.1f} weeks, and the '+
        f'typical (median) duration of unemployment is {median:.1f} weeks, as of {ldate}.')

write_txt(text_dir / 'unempdur.txt', text)

### Flows

In [ ]:
# Series stored as a dictionary
series = {'LNS17200000': 'NILF',
          'LNS17100000': 'UNEMP'}

# Start year and end year
dates = (1990, 2020)
data = bls_api(series, dates, bls_key)

df = data
df['TOTAL'] = data.astype('float').sum(axis=1)
sh = (df['NILF'] / df['TOTAL']).rename('total') * 100

sh.to_csv(data_dir / 'lf_flow.csv', index_label='date', header=True, float_format='%g')

sh.resample('QS').mean().rename('quarterly').to_csv(data_dir / 'lf_flow_q.csv', index_label='date', header=True)

totval = df['TOTAL'].iloc[-1] / 1000
nilfval = df['NILF'].iloc[-1] / 1000
unval = df['UNEMP'].iloc[-1] / 1000

shval = sh.iloc[-1]

sh3y = sh.iloc[-37]

ltdate = sh.index[-1].strftime('%B %Y')
yragodt = sh.index[-37].strftime('%B %Y')

text = (f'In {ltdate}, {totval:.1f} million people were newly employed (on a gross basis). '+
        f'Of these, {shval:.1f} percent were not looking for work in the prior '+
        'month. With low unemployment, new employees are being pulled '+
        'from outside of the labor force and bypassing unemployment. '+
        f'Three years ago, in {yragodt}, {sh3y:.1f} percent '+
        'of the newly employed were not looking for work month prior.')

write_txt(text_dir / 'lf_flow.txt', text)

### Wage Growth

In [ ]:
data1, data2 = pd.Series(), pd.Series()
columns = ['MONTH', 'YEAR', 'AGE', 'PWORWGT', 'WKWAGE', 'HRSUSL1', 'WORKFT']
for year in range(1989, 2020):
    df = (pd.read_feather(cps_dir / f'cps{year}.ft', columns=columns)
        .query('WKWAGE > 0 and WORKFT == 1'))
    data = df.groupby(['YEAR', 'MONTH']).apply(binned_wage)
    data.index = [pd.to_datetime(f'{ti[0]}-{ti[1]}-01') for ti in data.index]
    data1 = data1.append(data)
    
df = pd.DataFrame({'All': data1})

df.rolling(3).mean().to_csv(data_dir / 'uwe_bd.csv', index_label='date')
(df.pct_change(12).dropna() * 100).rolling(3).mean().to_csv(data_dir / 'uwe_bd_gr.csv', index_label='date')

# Series stored as a dictionary
series = {'LEU0252911200': 'value'}

# Start year and end year
dates = (2000, 2019)
df2 = bls_api(series, dates, bls_key)

df2.to_csv(data_dir / 'uwe_bls.csv', index_label='date')
(df2.pct_change(4).dropna() * 100).to_csv(data_dir / 'uwe_bls_gr.csv', index_label='date')

In [ ]:
(df.pct_change(12).dropna() * 100).rolling(3).mean()

### Average Hourly Earnings

In [ ]:
series = {'CES0500000003': 'ALL', 'CES0500000008': 'PNS'}
years = (1988, 2020)
df = bls_api(series, years, bls_key)
data = (df.pct_change(12) * 100).loc['1989':]
data.to_csv(data_dir / 'ahe.csv', index_label='date')

In [ ]:
ldate = dtxt(data.index[-1])['mon1']
alllt = data['ALL'].iloc[-1]
all_lt = f'{["increased" if alllt >= 0 else "decreased"][0]} by {abs(alllt):.1f} percent'
pnslt = data['PNS'].iloc[-1]
pns_lt = f'{["increased" if pnslt >= 0 else "decreased"][0]} by {abs(pnslt):.1f} percent'

s = series_info(df['ALL'])
all3m = (((s['last_3m'] / s['prev_3m'])**4) - 1) * 100
all_3m = f'{["increased" if all3m >= 0 else "decreased"][0]} at an annual rate of {abs(all3m):.1f} percent'

s = series_info(df['PNS'])
pns3m = (((s['last_3m'] / s['prev_3m'])**4) - 1) * 100
pns_3m = f'{["increased" if pns3m >= 0 else "decreased"][0]} at an annual rate of {abs(pns3m):.1f} percent'

In [ ]:
text = (f'Over the year ending {ldate}, nominal wages {all_lt} '+
        f'for all employees and {pns_lt} for production and '+
         'non-supervisory workers. Comparing the latest three months to the '+
        f'previous three months, nominal wages {all_3m} for all employees '+
        f'and {pns_3m} for production and non-supervisory employees.')

write_txt(text_dir / 'ahe_summary.txt', text)

In [ ]:
text

In [ ]:
series = {'CES3000000008': 'Manufacturing',
          'CES1000000008': 'Mining \& Logging',
          'CES4422000008': 'Utilities',
          'CES4142000008': 'Wholesale Trade',
          'CES5000000008': 'Information',
          'CES5500000008': 'Financial Activities',
          'CES6000000008': 'Professional \& Business Services',
          'CES6500000008': 'Education \& Health Services',
          'CES0500000008': 'Total Private',
          'CES2000000008': 'Construction',
          'CES7000000008': 'Leisure \& Hospitality',
          'CES4300000008': 'Transportation \& Warehousing',
          'CES4200000008': 'Retail Trade'}

years = (2017, 2020)
df = bls_api(series, years, bls_key)

In [ ]:
s = pd.read_csv(data_dir / 'cpi.csv')
allitems = s['ALL'].iloc[-1]
data = (df.pct_change(12).iloc[-1] * 100.0).sort_values(ascending=False)

(data.to_csv(data_dir / 'ahe_ind.csv', index_label='name', header=True))

write_txt(text_dir / 'ahe_bar_date.txt', df.index[-1].strftime('%B %Y'))

real = (data - allitems).drop('Total Private')
ltd = {i: (data.index[i].lower(), data.iloc[i]) for i in [0, 1, 2]}

txt1 = (f'By industry, {len(real.loc[real > 0])} of {len(real)} groups '+
         'experienced real wage growth (wage growth above the increase in '+
        f'prices indicated by the consumer price index). The {ltd[0][0]} '+
        f'industry had the fastest nominal growth rate, at {ltd[0][1]:.1f} percent, followed '+
        f'by {ltd[1][1]:.1f} percent in {ltd[1][0]} and {ltd[2][1]:.1f} percent in {ltd[2][0]}. ')

write_txt(text_dir / 'ahe_comp.txt', txt1)

text = ('\\noindent \hspace*{-2mm} \\begin{tikzpicture}'+
        '\\begin{axis}[\\barplotnogrid axis y line=left, \\barylab{4.0cm}{1.5ex}'+
        'width=5.2cm, bar width=1.8ex, height=7.2cm, xtick={0}, xmajorgrids,'+
        'enlarge y limits={abs=3mm}, enlarge x limits=0.1, '+
        f'\dbar{{x}}{{{allitems:.2f}}}, clip=false,'+
        'yticklabels from table={\\ahe}{name},'+
        'yticklabel style={font=\\footnotesize},'+
        'nodes near coords style={/pgf/number format/.cd, fixed zerofill,'+
        'precision=1, assume math mode}]'+
        '\\addplot[fill=blue!80!black, draw=none] '+
        'table [y expr=-\coordindex, x index=1] {\\ahe};'+
        f'\\node[right] at ({allitems:.2f}, -12.6) {{\\footnotesize \\textcolor{{black!80}}{{CPI}}}};'+
        '\end{axis}'+
        '\end{tikzpicture}\\\ '+
        '\\footnotesize{Source: Bureau of Labor Statistics} \hspace{32mm} \\tbllink{ahe_ind.csv}')

write_txt(text_dir / 'ahe_chart.txt', text)

In [ ]:
txt1

### CES data

In [ ]:
series = {'CES0000000001': 'ALL',
          'LNU00000000': 'TOT'}
years = (2011, 2020)
df = bls_api(series, years, bls_key)
data = df['ALL'].diff().loc['2014':]
data.to_csv(data_dir / 'nfp.csv', index_label='date', header='ALL')

In [ ]:
ldate = dtxt(data.index[-1])['mon1']
pdate = dtxt(data.index[-2])['mon1']

lval = data.iloc[-1]
pval = data.iloc[-2]
l3val = data.iloc[-3:].mean()

emp = df.loc['2012':, 'ALL'] * 1000

tot = df.loc['2012':, 'TOT']
final = emp - ((emp / tot).shift(1) * tot).dropna().rolling(12).mean()
final2 = (final.dropna() / 12).round(-3) / 1000
final2.to_csv(data_dir / 'nfp_pop.csv', index_label='date', header=['TOT'])

lpop = final2.iloc[-3:].mean()

In [ ]:
text = (f'The US economy added {lval:.0f},000 jobs in {ldate}, compared to '+
        f'{pval:.0f},000 in {pdate}, and an average of {l3val:.0f},000 over '+
        'the past three months. Over the same three months, the US needed to '+
        f'add an average of {lpop:.0f},000 jobs per month to maintain the prior '+
        'month employment rate.')

In [ ]:
write_txt(text_dir / 'nfp_basic_text.txt', text)